# BRCA Exchange Literature Search Export

Ingest artifacts from the pipeline, wrangle, and generate literature.json

In [2]:
import os
import json
import sqlite3
import pandas as pd

os.chdir(os.path.expanduser("~/data/pubmunch/crawl/"))

## Ingest
Ingest artifacts from various pieces of the pipeline indexed by normalized genomic hgvs and integer pmid 

In [3]:
connection = sqlite3.connect("file:text/articles.db?mode=ro", uri=True)
articles = pd.read_sql_query("SELECT * FROM articles", connection)
articles.pmid = articles.pmid.astype(str)
print("{} articles loaded from the articles sqlite database".format(articles.shape[0]))
articles.head()

15 articles loaded from the articles sqlite database


,articleId,externalId,source,publisher,origFile,journal,printIssn,eIssn,journalUniqueId,year,...,issue,page,pmid,pmcId,doi,fulltextUrl,time,offset,size,chunkId
0,5009042909,PMID9042909,,download,,American journal of human genetics,0002-9297,0002-9297,0370475,1997,...,3,505,9042909,1712523,10.1086/302525,https://linkinghub.elsevier.com/retrieve/pii/S...,2019-02-05T17:39:41+0000,78,22155,0_00000
1,5012473589,PMID12473589,,download,,Clinical cancer research : an official journal...,1078-0432,1078-0432,9502500,2002,...,12,3776,12473589,,,,2019-02-06T05:18:11+0000,78,72965,1_00000
2,5025192939,PMID25192939,,download,,Proceedings of the National Academy of Science...,0027-8424,1091-6490,7505876,2014,...,39,14205,25192939,4191771,10.1073/pnas.1415979111,,2019-02-06T05:18:36+0000,1329403691,68715,1_00000
3,5021987798,PMID21987798,,download,,Proceedings of the National Academy of Science...,0027-8424,1091-6490,7505876,2011,...,43,17773,21987798,3203756,10.1073/pnas.1110969108,,2019-02-06T05:18:57+0000,2729058227,97544,1_00000
4,5015994883,PMID15994883,,download,,Journal of medical genetics,0022-2593,1468-6244,2985087R,2005,...,7,602,15994883,1736090,10.1136/jmg.2004.024133,,2019-02-06T05:19:27+0000,4205419366,28415,1_00000


In [10]:
mentions = pd.read_csv("mentions-matched.tsv", sep="\t", encoding="utf-8", dtype="str")
print("Total matched mentions: {}".format(mentions.shape[0]))
mentions.head()

Total matched mentions: 8019


,pyhgvs_Genomic_Coordinate_38,pmid,snippets,score
0,chr17:g.43124027:ACT>A,11352856,previous history of breast or ovarian cancer. ...,3
1,chr17:g.43124027:ACT>A,11352856,All subjects were genotyped for the three muta...,3
2,chr17:g.43057062:T>TG,11352856,"miology, Biomarkers & Prevention Table 1 Age ...",3
3,chr17:g.43057059:T>TG,11352856,"miology, Biomarkers & Prevention Table 1 Age ...",3
4,chr13:g.32340526:AT>A,11352856,n cancer. All subjects were genotyped for the ...,2


In [9]:
print("Initial # mentions", mentions.shape[0])
pruned_mentions = mentions.drop_duplicates(["pyhgvs_Genomic_Coordinate_38", "pmid", "snippets"])
print("After dropping duplicates of pyhgvs_Genomic_Coordinate_38+pmid+snippets: {}".format(pruned_mentions.shape[0]))

Initial # mentions 8019
After dropping duplicates of pyhgvs_Genomic_Coordinate_38+pmid+snippets: 7919


In [15]:
# Some of the snippets are multiple mentions separated by | so unpack these,
# but limit to 3 as some have as many as 168!
print("Unpacking {} of {} snippets with multiple phrases seaparated by '|'".format(
    pruned_mentions[pruned_mentions.snippets.str.contains("|", regex=False)].shape[0], pruned_mentions.shape[0]))

# https://stackoverflow.com/questions/17116814/pandas-how-do-i-split-text-in-a-column-into-multiple-rows/21032532

# Reset index so each row id is unique vs. norm_g_hgvs
df = pruned_mentions.reset_index()
# df = variant_mentions[variant_mentions.snippet.str.contains("|", regex=False)].iloc[0:100].reset_index()

# Generate a new dataframe splitting each snippet segment into its own row
# Limit to max of 3 as some of them have > 100
snippets = df.apply(lambda x: pd.Series(x.snippets.split("|")[:3]), axis=1).stack()

# To line up with the original index
snippets.index = snippets.index.droplevel(-1)

# Join back to the original dataframe replaceing the old "snippet" columnd
snippets.name = "snippets"
del df["snippets"]
exploded = df.join(snippets).drop_duplicates(
    ["pyhgvs_Genomic_Coordinate_38", "snippets"]).set_index(
    "pyhgvs_Genomic_Coordinate_38", drop=True)
print("{} individual snippets after expanding and de-duplicating snippets".format(exploded.shape[0]))

exploded.head()

Unpacking 3931 of 7919 snippets with multiple phrases seaparated by '|'
13521 individual snippets after expanding and de-duplicating snippets


,index,pmid,score,snippets
pyhgvs_Genomic_Coordinate_38,,,,
chr17:g.43124027:ACT>A,0,11352856,3,previous history of breast or ovarian cancer. ...
chr17:g.43124027:ACT>A,0,11352856,3,ecific relative risks of breast cancer attribu...
chr17:g.43124027:ACT>A,0,11352856,3,"series, and individuals referred for prenatal ..."
chr17:g.43124027:ACT>A,1,11352856,3,All subjects were genotyped for the three muta...
chr17:g.43057062:T>TG,2,11352856,3,"miology, Biomarkers & Prevention Table 1 Age ..."


# Stats

In [33]:
print("Max mentions for a single variant: {} {}".format(
    exploded.groupby(["pyhgvs_Genomic_Coordinate_38"]).size().idxmax(),
    exploded.groupby(["pyhgvs_Genomic_Coordinate_38"]).size().max()))

print("Max mentions for a single paper from one variant: {} {}".format(
    exploded.groupby(["pyhgvs_Genomic_Coordinate_38", "pmid"]).size().idxmax(),
    exploded.groupby(["pyhgvs_Genomic_Coordinate_38", "pmid"]).size().max()))

print("Paper referenced from the most variants: {} {}".format(
    exploded.groupby(["pmid"]).size().idxmax(),
    exploded.groupby(["pmid"]).size().max()))

Max mentions for a single variant: chr17:g.43124027:ACT>A 62
Max mentions for a single paper from one variant: ('chr17:g.43057062:T>TG', '22430266') 11
Paper referenced from the most variants: 16397213 8978


In [34]:
print("Variant mention stats:")
print(exploded.reset_index().groupby(["pyhgvs_Genomic_Coordinate_38"])
      ["pyhgvs_Genomic_Coordinate_38"].count().describe())

Variant mention stats:
count    4587.000000
mean        2.947678
std         2.367346
min         1.000000
25%         1.000000
50%         3.000000
75%         4.000000
max        62.000000
Name: pyhgvs_Genomic_Coordinate_38, dtype: float64


In [35]:
print("Paper mentions stats:")
print(exploded.reset_index().groupby(["pmid"])
      ["pmid"].count().describe())

Paper mentions stats:
count      14.000000
mean      965.785714
std      2578.209521
min         2.000000
25%        10.750000
50%        17.500000
75%        23.250000
max      8978.000000
Name: pmid, dtype: float64


## Export

In [17]:
# Variants by pyhgvs_Genomic_Coordinate_38 by pmid with all snippets in a list
combined = exploded.groupby(["pyhgvs_Genomic_Coordinate_38", "pmid"])["snippets"].apply(lambda s: s.tolist())
print("Combined {} separate snippets down to {} after grouping by pmid".format(exploded.shape[0], combined.shape[0]))

Combined 13521 separate snippets down to 4736 after grouping by pmid


In [32]:
lit = {
    "date": open("date.txt").read().strip(),
    "papers": articles[articles.pmid.isin(pruned_mentions.pmid)].set_index("pmid", drop=False).to_dict(orient="index"),
    "variants": {
        k: {kk: vv[0] for kk, vv in v.unstack().transpose().iterrows()}
        for k, v in combined.groupby("pyhgvs_Genomic_Coordinate_38")},
}

with open("literature.json", "w") as output:
    output.write(json.dumps(lit, sort_keys=True))
    
print("Exported {} variants in {} papers".format(
    len(lit["variants"].keys()), len(lit["papers"].keys())))

Exported 4587 variants in 14 papers
